In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from os.path import exists
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from fastprogress import progress_bar
from IPython.core.debugger import set_trace
import scipy.stats as stats
from itertools import combinations
from scipy.stats import ttest_rel

from jsputils import classes

In [ ]:
loaddir = f'{os.getcwd()}/analysis_outputs/3-Encoding'
nc_loaddir = f'{os.getcwd()}/analysis_outputs/3c-NoiseCeilings'
figure_savedir = f'{os.getcwd()}/figure_outputs/Figure4-Encoding'

In [ ]:
subjs = [f'subj0{s}' for s in range(1,9)]
roi_list = ['FFA-1','FFA-2','OFA',
            'PPA','OPA',
            'EBA','FBA-1','FBA-2',
            'VWFA-1','VWFA-2','OWFA']#,'PPA','EBA','VWFA-1']
ncsnr_threshold = 0.3

test_imageset = 'special515'

model_names = ['alexnet-barlow-twins','alexnet-vggface','alexnet-barlow-twins-random','GistPC','Gabor']

domain_list = ['faces','bodies','objects','scenes','characters','layer']

# Define a dictionary for different ROIs and their respective domains
roi_dict = {'Face-selective ROIs': {'ROIs': ['OFA', 'FFA-1', 'FFA-2'], 'domain': 'faces'},
            'Body-selective ROIs': {'ROIs': ['FBA-1', 'FBA-2', 'EBA'], 'domain': 'bodies'},
            'Scene-selective ROIs': {'ROIs': ['OPA', 'PPA'], 'domain': 'scenes'},
            'Word-selective ROIs': {'ROIs': ['OWFA', 'VWFA-1', 'VWFA-2'], 'domain': 'characters'}}

domain_dict = {'OFA':'faces','FFA-1':'faces','FFA-2':'faces',
               'OPA':'scenes','PPA':'scenes',
               'EBA':'bodies','FBA-1':'bodies','FBA-2':'bodies',
               'OWFA':'characters', 'VWFA-1':'characters', 'VWFA-2':'characters'}

layer_list = ['conv3',
              'groupnorm3',
              'relu3',
              'conv4',
              'groupnorm4',
              'relu4',
              'conv5',
              'groupnorm5',
              'relu5',
              'maxpool5',
              'fc6',
              'batchnorm6',
              'relu6',
              'fc7',
              'batchnorm7',
              'relu7',
              'fc8',
              'batchnorm8']


In [ ]:
df_list = []

for roi in progress_bar(roi_list):
    for subj in subjs:
        for model_name in model_names:
            
            if 'alexnet' in model_name:
                for layer in np.flip(layer_list):
                    for domain in domain_list:
                        try:
                            fn = f'{loaddir}/{subj}_{roi}_nc-{ncsnr_threshold}_{model_name}_{layer}_{domain}.csv'
                            df_list.append(pd.read_csv(fn, index_col=None, header=0))
                            
                        except:
                            pass       
            else:
                try:
                    fn = f'{loaddir}/{subj}_{roi}_nc-{ncsnr_threshold}_{model_name}.csv'
                    df_list.append(pd.read_csv(fn, index_col=None, header=0))
                    
                except:
                    pass

combined_df = pd.concat(df_list, axis=0, ignore_index=True)


In [ ]:
combined_df

In [ ]:
# get reliability
noise_ceilings = []

for roi in progress_bar(roi_list):
    for subj in subjs:
        #print(roi, subj)
        
        savefn = f'{nc_loaddir}/GSN-NC_{roi}_{subj}_{test_imageset}_nc-{ncsnr_threshold}.npy'
        
        if exists(savefn):
            nc = np.load(savefn, allow_pickle=True).item()
            
            if len(nc['ncs']) > 1:
                this_ncs = np.mean(np.vstack(nc['ncs']),axis=0)
            else:
                this_ncs = np.array(nc['ncs'][0])
                
            #print(this_ncs)
            
            noise_ceilings.append([roi, subj, this_ncs[0], this_ncs[1]])
            
        else:
            print(savefn, 'does not exist')

noise_ceilings = pd.DataFrame(noise_ceilings, columns=['ROI', 'Subject', 'Univariate', 'RSA'])
noise_ceilings.rename(columns={'Univariate':'veUnivar','RSA':'veRSA'}, inplace=True)


In [ ]:
noise_ceilings[noise_ceilings['ROI'] == 'FBA-1']

# full layer summaries

In [ ]:
# Define the colors to be used for each domain
color_dict = {'faces': 'tomato', 'bodies': 'dodgerblue', 'objects': 'orange', 
              'scenes': 'limegreen', 'characters': 'purple', 'layer':'tomato'}

# Iterate over each unique model and outcome metric
for model_name in ['alexnet-barlow-twins','alexnet-vggface']:
    
    if 'barlow' in model_name:
        domains = ['faces','bodies','scenes','characters','objects']
        this_roi_list = ['FFA-1','PPA','EBA','VWFA-1']
        
    elif 'vggface' in model_name:
        domains = ['layer']
        this_roi_list = ['FFA-1','FFA-2','OFA','PPA','VWFA-1']
    
    for metric in ['veUnivar','veRSA']:
        
        print(model_name, metric)
        
        # Select the relevant columns from the dataframe
        df = copy.deepcopy(combined_df)
        
        # Filter the data for the specific model and metric
        df = df[(df['model_name'] == model_name) & (df['partition'] == 'test')]
        
        # Initialize the figure and subplots
        plt.figure(figsize=(30,6))
        c=1
        #fig, axs = plt.subplots(11, 1, figsize=(16, 48), sharex=True)
        plt.title(f'{model_name} - {metric}', fontsize=16)
        
        # Iterate over each ROI and create a subplot
        for i, roi in enumerate(this_roi_list):
            
            plt.subplot(1,len(this_roi_list),c)
            
            # Filter the data for the specific ROI
            roi_df = df[df['ROI'] == roi]
            
            numeric_cols = roi_df.select_dtypes(include=np.number).columns.tolist()
            grouped = roi_df.groupby(['domain', 'layer'], sort=False)[numeric_cols].agg(['mean', 'sem'])
                        
            for domain in domains:
                color = color_dict[domain]
        
                try:
                    # Get the data for this domain
                    domain_data = grouped.loc[domain]
                except:
                    set_trace()

                # Convert the layer index to a numpy array and flip it
                x = np.flip(domain_data.index.to_numpy())

                # Get the mean and SE for this metric, flip them
                y = np.flip(domain_data[(metric, 'mean')].to_numpy())
                yerr = np.flip(domain_data[(metric, 'sem')].to_numpy())

                # Plot the mean line
                plt.plot(x, y, color=color, label=domain, marker='.')

                # Add the shaded region for the SE
                plt.fill_between(x, y - yerr, y + yerr, color=color, alpha=0.2)

                
            ncs = []

            for subj in subjs:
                
                if 'Univar' in metric:
                    ncs.append(noise_ceilings[np.logical_and(noise_ceilings['ROI'] == roi,
                               noise_ceilings['Subject'] == subj)]['veUnivar'].values)
                elif 'RSA' in metric:
                    ncs.append(noise_ceilings[np.logical_and(noise_ceilings['ROI'] == roi,
                               noise_ceilings['Subject'] == subj)]['veRSA'].values)

            ncs = np.concatenate(ncs)

            nc_range = [np.nanmin(ncs), np.nanmax(ncs)]

            plt.fill_between(np.arange(-0.5,len(layer_list)+0.5), nc_range[0], nc_range[1], color='gray',alpha=0.2)
                
            # Set the subplot properties
            plt.title(f'{roi} {metric}')
            plt.xticks(rotation=90)
            plt.ylabel('Prediction Levels')
            plt.grid(True)
            plt.ylim([-0.4,1])
            plt.plot(np.arange(len(layer_list)), np.zeros((len(layer_list),)),'k',linewidth=2)
            #plt.legend()
            c+=1
            
            if roi == 'VWFA-1' and 'vggface' in model_name:
                break
            elif roi == 'OPA':
                c+=1
                
        # Set the x-axis label for the last subplot
        plt.xlabel('Model Layers')
        
        # Adjust the spacing between subplots
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        
    # Show the figure
    plt.show()

In [ ]:
def find_max_score_layers(results_df, metric):
    
    # Add a 'row_order' column to the original dataframe
    results_df['row_order'] = np.arange(len(results_df))

    valid_models = ['alexnet-barlow-twins', 'alexnet-barlow-twins-random', 'alexnet-vggface', 'GistPC', 'Gabor']
    results_df = results_df.fillna(-999)

    df_max = pd.DataFrame()
    for model_name in valid_models:
        if model_name in ['alexnet-barlow-twins', 'alexnet-barlow-twins-random', 'alexnet-vggface']:
            val_df = results_df[(results_df['partition'] == 'val') & (results_df['model_name'] == model_name)]
            max_layers = val_df.groupby(['subj', 'ROI', 'model_name', 'domain'])[metric].idxmax().apply(lambda x: val_df.loc[x, 'layer'])

            for (subj, ROI, model, domain), layer in max_layers.items():
                max_df = results_df[(results_df['subj'] == subj) & 
                                    (results_df['ROI'] == ROI) &
                                    (results_df['model_name'] == model) &
                                    (results_df['layer'] == layer) &
                                    (results_df['domain'] == domain) &
                                    (results_df['partition'] == 'test')]

                df_max = pd.concat([df_max, max_df], axis=0)
        else:
            max_df = results_df[(results_df['model_name'] == model_name) & (results_df['partition'] == 'test')]
            df_max = pd.concat([df_max, max_df], axis=0)
        
    # Sort df_max based on 'row_order' column
    df_max = df_max.sort_values('row_order')

    # Remove the 'row_order' column
    df_max = df_max.drop(columns=['row_order'])

    return df_max


In [ ]:
df_max = dict()

metrics = ['veUnivar','veRSA','cUnivar','cRSA']

for metric in progress_bar(metrics):
    df_max[metric] = find_max_score_layers(copy.deepcopy(combined_df), metric)


In [ ]:

def pairwise_ttest_with_first(model_data, alpha=0.01):
    n = len(model_data)
    significant_pairs = []
    p_values = np.ones(n)
    for i in range(1, n):
        _, p = ttest_rel(model_data[0], model_data[i])
        p_values[i] = p
        if p < alpha:
            significant_pairs.append((0, i))
    return significant_pairs, p_values


# main figure 4

In [ ]:
metrics = ['veUnivar','veRSA']

n_tests = 36 # (3 * 4) + (8 * 3)

ttest_alpha = 0.05 / n_tests # bonferroni

# Define the models and their corresponding colors
model_colors = {'face':
                {'alexnet-barlow-twins': 'dodgerblue',
                 'alexnet-vggface':'red',
                    'alexnet-barlow-twins-random': 'lightgray',
                    'GistPC': 'violet',
                    'Gabor': 'tan'},
                'non-face':{
                        'alexnet-barlow-twins': 'dodgerblue',
                        'alexnet-barlow-twins-random': 'lightgray',
                        'GistPC': 'violet',
                        'Gabor': 'tan'}}

for metric in metrics:
    
    # Select the relevant columns from the dataframe
    df = copy.deepcopy(df_max[metric])

    df_roi = df[['subj', 'ROI', 'model_name', 'domain', 'veUnivar', 'veRSA']]

    # Initialize the figure and subplots
    fig, axs = plt.subplots(1, 11, figsize=(20, 8), sharey=True)
    axs = axs.flatten()

    # Iterate over each ROI
    for i, (roi, ax) in enumerate(zip(df_roi['ROI'].unique(), axs)):
        # Filter the dataframe for the current ROI
        df_roi_sub = df_roi[df_roi['ROI'] == roi]

        # Initialize a list to store the data for each model
        model_data = []

        # Iterate over each model and retrieve the data for the current ROI
        if 'FFA' in roi or 'OFA' in roi:
            these_colors = model_colors['face']
        else:
            these_colors = model_colors['non-face']
            
        for model, color in these_colors.items():
            if 'barlow-twins' in model:
                model_data.append(df_roi_sub[np.logical_and(df_roi_sub['model_name'] == model,
                                                            df_roi_sub['domain'] == domain_dict[roi])][metric].values)
                
                if 'random' not in model:
                    if 'FFA' in roi or 'OFA' in roi or roi == 'PPA' or roi == 'EBA' or roi == 'VWFA-1':
                        print(roi, model, metric, np.nanmean(model_data[-1]), np.nanstd(model_data[-1]))
            else:
                model_data.append(df_roi_sub[df_roi_sub['model_name'] == model][metric].values)
                
                if 'vggface' in model:
                    print(roi, model, metric, np.nanmean(model_data[-1]), np.nanstd(model_data[-1]))
            
        # Create the violin plots for the models
        sns.violinplot(data=model_data, ax=ax, palette=these_colors.values(), 
                       inner='point',linewidth=1,scale='width',width=0.6,cut=1.5)#,gridsize=10)

        ncs = []

        for subj in subjs:

            ncs.append(noise_ceilings[np.logical_and(noise_ceilings['ROI'] == roi,
                       noise_ceilings['Subject'] == subj)][metric].values)

        ncs = np.concatenate(ncs)
        
        nc_range = [np.nanmin(ncs), np.nanmax(ncs)]
        
        n_models = len(model_data)

        ax.fill_between(np.arange(-0.5,n_models+0.5), nc_range[0], nc_range[1], color='gray',alpha=0.2,zorder=0)

        # Set the title as the ROI name
        ax.set_title(roi)
        ax.set_ylim([-0.4,1.2])

        ax.hlines(np.arange(-0.2,1.2,0.2),-0.5,n_models-0.5,'k',linewidth=0.25)
        ax.hlines(0,-0.5,n_models-0.5,'k',linewidth=1,zorder=0)
        
        # Call pairwise_wilcoxon to get significant pairs
        significant_pairs, p_values = pairwise_ttest_with_first(model_data, alpha = ttest_alpha)
        
        #print(np.mean(p_values[-3:]))
        # Draw lines for significant pairs
        line_y = 0.97  # Initial y-position for lines
        for pair in significant_pairs:
            ax.plot(pair, [line_y, line_y], color='black', lw=0.75)
            line_y -= 0.015  # Decrement y-position for next line

        plt.xticks(rotation=90,fontsize=8);
        ax.set_xlabel('')
        ax.axis('off')
        
    plt.tight_layout()
    plt.savefig(f"{figure_savedir}/encoding-best-layers-{metric}.tiff")
    plt.close()


In [ ]:
loaddir_ols = f'{os.getcwd()}/analysis_outputs/3-Encoding/alexnet-barlow-twins-ols'

this_roi_list = ['FFA-1','PPA','EBA','VWFA-1']

df_list = []

for lddir in [loaddir, loaddir_ols]:
    for roi in progress_bar(this_roi_list):
        for subj in subjs:
            for model_name in ['alexnet-barlow-twins']:
                for layer in np.flip(layer_list):
                    for domain in domain_list:
                        try:
                            fn = f'{lddir}/{subj}_{roi}_nc-{ncsnr_threshold}_{model_name}_{layer}_{domain}.csv'
                            df_list.append(pd.read_csv(fn, index_col=None, header=0))

                        except:
                            pass

combined_df_ols = pd.concat(df_list, axis=0, ignore_index=True)


In [ ]:
combined_df_ols

In [ ]:
model_name = 'alexnet-barlow-twins'
domains = ['faces','bodies','scenes','characters','objects']
ft = 24

fg = 0
for analysis in ['Univar','RSA']:

    metric_methods = [(f've{analysis}','ols'),
                      (f've{analysis}','lasso')]

    metric_labels = [f'voxel-encoding {analysis} (OLS)',
                     f'voxel-encoding {analysis} (sparse positive)']

    # Iterate over each ROI and create a subplot
    for i, roi in enumerate(this_roi_list):

        for m, metric_method in enumerate(metric_methods):
            
            # Initialize the figure and subplots
            plt.figure(figsize=(16,7))

            metric, method = metric_method

            # Select the relevant columns from the dataframe
            df = copy.deepcopy(combined_df_ols)

            # Filter the data for the specific model and metric
            df = df[(df['model_name'] == model_name) & (df['partition'] == 'val')]

            roi_df = df[(df['ROI'] == roi) & (df['method'] == method)]

            numeric_cols = roi_df.select_dtypes(include=np.number).columns.tolist()
            grouped = roi_df.groupby(['domain', 'layer'], sort=False)[numeric_cols].agg(['mean', 'sem'])

            for domain in domains:
                color = color_dict[domain]

                # Get the data for this domain
                domain_data = grouped.loc[domain]

                # Convert the layer index to a numpy array and flip it
                x = np.flip(domain_data.index.to_numpy())

                # Get the mean and SE for this metric, flip them
                y = np.flip(domain_data[(metric, 'mean')].to_numpy())
                yerr = np.flip(domain_data[(metric, 'sem')].to_numpy())

                if roi == 'FFA-1' and domain == 'faces':
                    lw = 4
                    zo = 10
                elif roi == 'PPA' and domain == 'scenes':
                    lw = 4
                    zo = 10
                elif roi == 'EBA' and domain == 'bodies':
                    lw = 4
                    zo = 10
                elif roi == 'VWFA-1' and domain == 'characters':
                    lw = 4
                    zo = 10
                else:
                    lw = 1.5
                    zo = 0
                    
                # Plot the mean line
                plt.plot(x, y, color=color, label=domain, marker='.',linewidth=lw,zorder=zo)

                # Add the shaded region for the SE
                plt.fill_between(x, y - yerr, y + yerr, color=color, 
                                 alpha=0.2,linewidth=0,zorder=zo)

            ncs = []

            for subj in subjs:

                if 'Univar' in metric:
                    ncs.append(noise_ceilings[np.logical_and(noise_ceilings['ROI'] == roi,
                               noise_ceilings['Subject'] == subj)]['veUnivar'].values)
                elif 'RSA' in metric:
                    ncs.append(noise_ceilings[np.logical_and(noise_ceilings['ROI'] == roi,
                               noise_ceilings['Subject'] == subj)]['veRSA'].values)

            ncs = np.concatenate(ncs)

            nc_range = [np.nanmin(ncs), np.nanmax(ncs)]

            plt.fill_between(np.arange(-0.5,len(layer_list)+0.5), nc_range[0], nc_range[1], 
                             color='gray',alpha=0.2)

            plt.xticks(rotation=90,fontsize=ft)
            plt.grid('on')
            # get rid of the frame
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            plt.yticks(fontsize=ft)
            plt.legend(fontsize=ft-3)
            plt.ylim([-0.25,0.85])
            plt.plot(np.arange(len(layer_list)), np.zeros((len(layer_list),)),'k',linewidth=2)
            plt.tight_layout()
            plt.savefig(f"{figure_savedir}/encoding_summary-{fg}-{roi}-{metric_labels[m]}.tiff")
            plt.close()
            fg+=1

# which layers are most predictive?


In [ ]:
for model_name in ['alexnet-barlow-twins', 'alexnet-barlow-twins-random','alexnet-vggface']:
    
    if 'vggface' in model_name:
                
        this_layer_list = ['conv3','relu3',
                      'conv4','relu4',
                      'conv5','relu5','maxpool5',
                      'fc6','relu6',
                      'fc7','relu7',
                      'fc8']
        
    else:
        this_layer_list = copy.deepcopy(layer_list)

    for metric in metrics:
        
        print(model_name, metric)

        # Create a copy of the dataframe for 'alexnet-barlow-twins' model only
        this_df_max = df_max[metric]
        this_df_max = this_df_max[this_df_max['model_name'] == model_name].copy()
        
        # Convert layer names to numbers
        layer_mapping = {name: i for i, name in enumerate(this_layer_list)}
        this_df_max['layer_num'] = this_df_max['layer'].map(layer_mapping)

        # Initialize a figure
        fig, axs = plt.subplots(1, 4, figsize=(12, 4), sharey=True)

        for ax, (roi_type, roi_data) in zip(axs, roi_dict.items()):
            # Extract ROI and domain information
            roi_list = roi_data['ROIs']
            
            if 'vggface' in model_name:
                domain = 'layer'
                
            else:
                domain = roi_data['domain']

            # Filter data for the particular ROI type, ROI list, and domain
            df_roi = this_df_max[this_df_max['ROI'].isin(roi_list) & (this_df_max['domain'] == domain)].copy()

            # Sort the DataFrame based on the custom order of ROIs
            df_roi = df_roi.sort_values('ROI', key=lambda x: x.map({roi: i for i, roi in enumerate(roi_list)}))

            #if 'FFA-1' in roi_list and 'vgg' in model_name:
            #    assert(1==2)
            # Plot data for each subject separately
            for subj, subj_df in df_roi.groupby('subj'):

                x = []
                y = []
                for r in range(len(roi_list)):
                    if roi_list[r] in list(subj_df['ROI'].values):
                        x.append(r)
                        y.append(subj_df[subj_df['ROI'] == roi_list[r]]['layer_num'].values[0] + np.random.normal(0,0.1))
                    else:
                        x.append(np.nan)
                        y.append(np.nan)

                ax.plot(np.array(x) + np.random.normal(0,0.01,size=len(x)),y, marker='o', linestyle='-', linewidth=4, markersize=5,label=subj, alpha=0.8)

            #ax.set_title(roi_type)
            ax.set_xticks(range(len(roi_list)))  # Set the x-ticks based on the roi_list order
            ax.set_xticklabels(roi_list,fontsize=12)  # Set the x-tick labels as roi_list
            ax.set_yticks(range(len(this_layer_list)))
            ax.set_yticklabels(this_layer_list,fontsize=12)
            ax.grid(True)

        # Move the legend out of the main plot
        #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.tight_layout()
        plt.savefig(f"{figure_savedir}/top-layer-idx-{model_name}-{metric}.tiff")
        plt.close()


